# Load goodreads data
Og lidt oprydning.

In [ ]:
import main
import pandas as pd

# load goodreads data, clean whitespace from author names and filter out unread books
df = main.read_data()
df['Author'] = df['Author'].str.replace(r'\s+', ' ', regex=True)
df['Author'].unique()
df=df[df['Read Count'] > 0]

# collapse the list on author and add book count and average 'Average Rating'
df = df.groupby('Author').apply(lambda x: pd.Series({
    'Title': ', '.join(x['Title']),
    'Average Rating': x['Average Rating'].mean(),
})).reset_index()

df['Books'] = df['Title'].str.split(', ')
df.drop('Title', axis=1, inplace=True)

# Author country lookup


## Funktion til at kalde wiki datas SPARQL endpoint


In [ ]:
import sys
import re
from SPARQLWrapper import SPARQLWrapper, JSON

def execute_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s (christian@dalager.com)" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


def get_author_countries(author_name):
    author_name = re.sub(r"\s+", " ", author_name)
    author_name = author_name.strip()

    endpoint_url = "https://query.wikidata.org/sparql"

    query = (
        """SELECT ?countryLabel WHERE {
        ?author ?label "%s"@en.
        ?author wdt:P27 ?country.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        }"""
        % author_name
    )

    results = execute_query(endpoint_url, query)

    bindings = results["results"]["bindings"]
    countries = list({entry["countryLabel"]["value"] for entry in bindings})

    return countries


## Kald data apiet
Det tager 1-2 minutter at lave 369 opslag

In [ ]:
# extract all author names from "Author" to a unique list
import pandas as pd
authors = df['Author'].unique()

print(f'Requesting {len(authors)} authors')

# Create a list to collect author and their countries
author_country_list = []

# Iterate over all authors and get the countries  (1.5 min)
for author in authors:
    countries = get_author_countries(author)  # Make sure this returns a list of countries
    author_country_list.append({'Author': author, 'Countries': countries})

df_authorcountries = pd.DataFrame(author_country_list)
    
# save to csv
df_authorcountries.to_csv('../data/author_countries.csv', index=False)


# Begynd at arbejde med listerne

In [ ]:
df_authorcountries = pd.read_csv('../data/author_countries.csv')

import ast
for index, row in df_authorcountries.iterrows():
    countries = ast.literal_eval(row['Countries'])
    df_authorcountries.at[index, 'Countries'] = countries

# Manually located authors' countries in the dataset

In [ ]:
df_manually_placed = pd.read_csv('../data/manually_placed.csv')

for index, row in df_manually_placed.iterrows():
    countries = ast.literal_eval(row['Countries'])
    df_manually_placed.at[index, 'Countries'] = countries

df_manually_placed.rename(columns={'Countries': 'Countries_Manually_Placed'}, inplace=True)
merged_df = pd.merge(df_authorcountries, df_manually_placed[['Author', 'Countries_Manually_Placed']], on='Author', how='left', validate='1:1')
# Replace the 'Countries' column with the corrected countries where available
merged_df['Countries'] = merged_df['Countries_Manually_Placed'].fillna(merged_df['Countries'])

# Drop the 'Countries_Manually_Placed' column if you don't need it anymore
merged_df=merged_df.drop(columns=['Countries_Manually_Placed'])

df_authorcountries = merged_df


In [ ]:
df_authorcountries





# Complete the final dataset

In [ ]:
df_final = pd.merge(df, df_authorcountries, on='Author', how='left', validate='1:1')


# Define a mapping from old country names to new country names
country_mapping = {
    'Ancient Rome': 'Italy',
    'Austria-Hungary': 'Austria',
    'Cisleithania': 'Austria',
    'Classical Athens': 'Greece',
    'Czechoslovakia': 'Czech Republic',
    'Dutch Republic': 'Netherlands',
    'England': 'United Kingdom',
    'German Empire': 'Germany',
    'Kingdom of Denmark': 'Denmark',
    'Kingdom of England': 'United Kingdom',
    'Kingdom of Great Britain': 'United Kingdom',
    'Kingdom of Italy': 'Italy',
    'Kingdom of the Netherlands': 'Netherlands',
    'Nazi Germany': 'Germany',
    'Russian Empire': 'Russia',
    'Russian Socialist Federative Soviet Republic': 'Russia',
    'Russian Soviet Federative Socialist Republic': 'Russia',
    'Scotland': 'United Kingdom',
    'United Kingdom of Great Britain and Ireland': 'United Kingdom',
    'Ireland':'Republic of Ireland',
    'Weimar Republic': 'Germany',
    'statelessness': None
    # Add more mappings as needed
}

# Function to clean country names
def clean_country_list(country_list):
    replaced= [country_mapping.get(country, country) for country in country_list if country_mapping.get(country, country) is not None]
    return list(set(replaced))

# Apply the function to the 'Countries' column
df_final['Countries'] = df_final['Countries'].apply(clean_country_list)


df_final.to_csv('../data/myauthorlist_with_countries_and_books.csv', index=False)

In [ ]:
df_final

# Show me the countries!

In [ ]:
# list of uniqe countries
cs=list(df_final['Countries'])
cs = {item for sublist in cs for item in sublist}
cs = list(cs)
cs.sort()
cs


# Barchart

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

# Aggregate the data
country_count = defaultdict(int)
for countries in df_final['Countries']:
    for country in countries:
        country_count[country] += 1

# Convert to DataFrame for plotting
country_df = pd.DataFrame(list(country_count.items()), columns=['Country', 'Count'])

# Bar Chart
plt.figure(figsize=(14, 7))
sns.barplot(x='Count', y='Country', data=country_df.sort_values(by='Count', ascending=False))
plt.title('Forfattere læst fordelt på lande')
plt.xlabel('Antal')
plt.ylabel('Land')
plt.show()